In [ ]:
import os
import json
import time
import zipfile
import datetime
import warnings
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from tqdm import tqdm
from collections import defaultdict

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torch.optim.lr_scheduler import ReduceLROnPlateau

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor

import optuna

In [ ]:
class Utility:
    @staticmethod
    def add_noise(data, mask, ng, pixel_size=2.):
        """
        Add noise to a noiseless convergence map.

        Parameters
        ----------
        data : np.array
            Noiseless convergence maps.
        mask : np.array
            Binary mask map.
        ng : float
            Number of galaxies per arcmin². This determines the noise level; a larger number means smaller noise.
        pixel_size : float, optional
            Pixel size in arcminutes (default is 2.0).
        """

        return data + np.random.randn(*data.shape) * 0.4 / (2*ng*pixel_size**2)**0.5 * mask
    
    @staticmethod
    def load_np(data_dir, file_name):
        file_path = os.path.join(data_dir, file_name)
        return np.load(file_path)

    @staticmethod
    def save_np(data_dir, file_name, data):
        file_path = os.path.join(data_dir, file_name)
        np.save(file_path, data)

    @staticmethod
    def save_json_zip(submission_dir, json_file_name, zip_file_name, data):
        """
        Save a dictionary with 'means' and 'errorbars' into a JSON file,
        then compress it into a ZIP file inside submission_dir.

        Parameters
        ----------
        submission_dir : str
            Path to the directory where the ZIP file will be saved.
        file_name : str
            Name of the ZIP file (without extension).
        data : dict
            Dictionary with keys 'means' and 'errorbars'.

        Returns
        -------
        str
            Path to the created ZIP file.
        """
        os.makedirs(submission_dir, exist_ok=True)

        json_path = os.path.join(submission_dir, json_file_name)

        # Save JSON file
        with open(json_path, "w") as f:
            json.dump(data, f)

        # Path to ZIP
        zip_path = os.path.join(submission_dir, zip_file_name)

        # Create ZIP containing only the JSON
        with zipfile.ZipFile(zip_path, "w", compression=zipfile.ZIP_DEFLATED) as zf:
            zf.write(json_path, arcname=json_file_name)

        # Remove the standalone JSON after zipping
        os.remove(json_path)

        return zip_path

In [ ]:
class Data:
    def __init__(self, data_dir, USE_PUBLIC_DATASET):
        self.USE_PUBLIC_DATASET = USE_PUBLIC_DATASET
        self.data_dir = data_dir
        self.mask_file = 'WIDE12H_bin2_2arcmin_mask.npy'
        self.viz_label_file = 'label.npy'
        if self.USE_PUBLIC_DATASET:
            self.kappa_file = 'WIDE12H_bin2_2arcmin_kappa.npy'
            self.label_file = self.viz_label_file
            self.Ncosmo = 101  # Number of cosmologies in the entire training data
            self.Nsys = 256    # Number of systematic realizations in the entire training data
            self.test_kappa_file = 'WIDE12H_bin2_2arcmin_kappa_noisy_test.npy'
            self.Ntest = 4000  # Number of instances in the test data
        else:
            self.kappa_file = 'sampled_WIDE12H_bin2_2arcmin_kappa.npy'
            self.label_file = 'sampled_label.npy'
            self.Ncosmo = 3    # Number of cosmologies in the sampled training data
            self.Nsys = 30     # Number of systematic realizations in the sampled training data
            self.test_kappa_file = 'sampled_WIDE12H_bin2_2arcmin_kappa_noisy_test.npy'
            self.Ntest = 3     # Number of instances in the sampled test data
        
        self.shape = [1424,176] # dimensions of each map 
        self.pixelsize_arcmin = 2 # pixel size in arcmin
        self.pixelsize_radian = self.pixelsize_arcmin / 60 / 180 * np.pi # pixel size in radian
        self.ng = 30  # galaxy number density. This determines the noise level of the experiment. Do not change this number.

    def load_train_data(self):
        self.mask = Utility.load_np(data_dir=self.data_dir, file_name=self.mask_file) # A binary map that shows which parts of the sky are observed and which areas are blocked
        self.kappa = np.zeros((self.Ncosmo, self.Nsys, *self.shape), dtype=np.float16)
        self.kappa[:,:,self.mask] = Utility.load_np(data_dir=self.data_dir, file_name=self.kappa_file) # Training convergence maps
        self.label = Utility.load_np(data_dir=self.data_dir, file_name=self.label_file) # Training labels (cosmological and physical paramameters) of each training map
        self.viz_label = Utility.load_np(data_dir=self.data_dir, file_name=self.viz_label_file) # For visualization of parameter distributions

    def load_test_data(self):
        self.kappa_test = np.zeros((self.Ntest, *self.shape), dtype=np.float16)
        self.kappa_test[:,self.mask] = Utility.load_np(data_dir=self.data_dir, file_name=self.test_kappa_file) # Test noisy convergence maps

In [ ]:
class Visualization:
    
    @staticmethod
    def plot_mask(mask):
        plt.figure(figsize=(30,100))
        plt.imshow(mask.T)
        plt.show()

    @staticmethod
    def plot_noiseless_training_convergence_map(kappa):
        plt.figure(figsize=(30,100))
        plt.imshow(kappa[0,0].T, vmin=-0.02, vmax=0.07)
        plt.show()

    @staticmethod
    def plot_noisy_training_convergence_map(kappa, mask, pixelsize_arcmin, ng):
        plt.figure(figsize=(30,100))
        plt.imshow(Utility.add_noise(kappa[0,0], mask, ng, pixelsize_arcmin).T, vmin=-0.02, vmax=0.07)
        plt.show()

    @staticmethod
    def plot_cosmological_parameters_OmegaM_S8(label):
        plt.scatter(label[:,0,0], label[:,0,1])
        plt.xlabel(r'$\Omega_m$')
        plt.ylabel(r'$S_8$')
        plt.show()

    @staticmethod
    def plot_baryonic_physics_parameters(label):
        plt.scatter(label[0,:,2], label[0,:,3])
        plt.xlabel(r'$T_{\mathrm{AGN}}$')
        plt.ylabel(r'$f_0$')
        plt.show()

    @staticmethod
    def plot_photometric_redshift_uncertainty_parameters(label):
        plt.hist(label[0,:,4], bins=20)
        plt.xlabel(r'$\Delta z$')
        plt.show()

In [ ]:
class Score:
    @staticmethod
    def _score_phase1(true_cosmo, infer_cosmo, errorbar):
        """
        Computes the log-likelihood score for Phase 1 based on predicted cosmological parameters.

        Parameters
        ----------
        true_cosmo : np.ndarray
            Array of true cosmological parameters (shape: [n_samples, n_params]).
        infer_cosmo : np.ndarray
            Array of inferred cosmological parameters from the model (same shape as true_cosmo).
        errorbar : np.ndarray
            Array of standard deviations (uncertainties) for each inferred parameter 
            (same shape as true_cosmo).

        Returns
        -------
        np.ndarray
            Array of scores for each sample (shape: [n_samples]).
        """
        
        sq_error = (true_cosmo - infer_cosmo)**2
        scale_factor = 1000  # This is a constant that scales the error term.
        score = - np.sum(sq_error / errorbar**2 + np.log(errorbar**2) + scale_factor * sq_error, 1)
        score = np.mean(score)
        if score >= -10**6: # Set a minimum of the score (to properly display on Codabench)
            return score
        else:
            return -10**6

In [ ]:
root_dir = os.getcwd()
print("Root directory is", root_dir)

In [ ]:
# USE_PUBLIC_DATASET = False

USE_PUBLIC_DATASET = True
PUBLIC_DATA_DIR = '../data'  # This is only required when you set USE_PUBLIC_DATASET = True

In [ ]:
if not USE_PUBLIC_DATASET:                                         # Testing this startking kit with a tiny sample of the training data (3, 30, 1424, 176)
    DATA_DIR = os.path.join(root_dir, 'input_data/')
else:                                                              # Training your model with all training data (101, 256, 1424, 176)
    DATA_DIR = PUBLIC_DATA_DIR    

In [ ]:
# Initialize Data class object
data_obj = Data(data_dir=DATA_DIR, USE_PUBLIC_DATASET=USE_PUBLIC_DATASET)

# Load train data
data_obj.load_train_data()

# Load test data
data_obj.load_test_data()

In [ ]:
Ncosmo = data_obj.Ncosmo
Nsys = data_obj.Nsys

print(f'There are {Ncosmo} cosmological models, each has {Nsys} realizations of nuisance parameters in the training data.')

In [ ]:
print(f'Shape of the training data = {data_obj.kappa.shape}')
print(f'Shape of the mask = {data_obj.mask.shape}')
print(f'Shape of the training label = {data_obj.label.shape}')
print(f'Shape of the test data = {data_obj.kappa_test.shape}')

In [ ]:
# import gc

# np.random.seed(31415)
# Ncosmo = data_obj.kappa.shape[0]
# Nsys = data_obj.kappa.shape[1]

# # Initialize output array
# noisy_kappa = np.zeros(data_obj.kappa.shape, dtype=np.float32)

# # Process one cosmology at a time to minimize memory usage
# for cosmo_idx in tqdm(range(Ncosmo), desc="Adding noise"):
#     # Process this entire cosmology
#     cosmo_data = data_obj.kappa[cosmo_idx].astype(np.float64)
    
#     noisy_kappa[cosmo_idx] = Utility.add_noise(
#         data=cosmo_data,
#         mask=data_obj.mask,
#         ng=data_obj.ng,
#         pixel_size=data_obj.pixelsize_arcmin
#     ).astype(np.float32)
    
#     # Clean up
#     del cosmo_data
    
#     # Force garbage collection every 10 cosmologies
#     if (cosmo_idx + 1) % 10 == 0:
#         gc.collect()

# print("Noise addition complete!")

In [ ]:
# print(f'Shape of the noised data {noisy_kappa.shape}')

In [ ]:
# # Split the data into training and validation sets

# NP_idx = np.arange(Nsys)  # The indices of Nsys nuisance parameter realizations
# split_fraction = 0.2      # Set the fraction of data you want to split (between 0 and 1)
# seed = 5566               # Define your random seed for reproducible results

# train_NP_idx, val_NP_idx = train_test_split(NP_idx, test_size=split_fraction,
#                                             random_state=seed)

# noisy_kappa_train = noisy_kappa[:, train_NP_idx]      # shape = (Ncosmo, len(train_NP_idx), 1424, 176)
# label_train = data_obj.label[:, train_NP_idx]         # shape = (Ncosmo, len(train_NP_idx), 5)
# noisy_kappa_val = noisy_kappa[:, val_NP_idx]          # shape = (Ncosmo, len(val_NP_idx), 1424, 176)
# label_val = data_obj.label[:, val_NP_idx]             # shape = (Ncosmo, len(val_NP_idx), 5)

# Ntrain = label_train.shape[0]*label_train.shape[1]
# Nval = label_val.shape[0]*label_val.shape[1]

In [ ]:
# print(f'Shape of the split training data = {noisy_kappa_train.shape}')
# print(f'Shape of the split validation data = {noisy_kappa_val.shape}')

# print(f'Shape of the split training labels = {label_train.shape}')
# print(f'Shape of the split validation labels = {label_val.shape}')

In [ ]:
# # Save the split data and labels for future usage

# Utility.save_np(data_dir=DATA_DIR, file_name="noisy_kappa_train.npy",data=noisy_kappa_train)
# Utility.save_np(data_dir=DATA_DIR, file_name="label_train.npy",data=label_train)
# Utility.save_np(data_dir=DATA_DIR, file_name="noisy_kappa_val.npy",data=noisy_kappa_val)
# Utility.save_np(data_dir=DATA_DIR, file_name="label_val.npy",data=label_val)

In [ ]:
# Load the saved split data (if you saved it at DATA_DIR before)

noisy_kappa_train = Utility.load_np(data_dir=DATA_DIR, file_name="noisy_kappa_train.npy")
label_train = Utility.load_np(data_dir=DATA_DIR, file_name="label_train.npy")
noisy_kappa_val = Utility.load_np(data_dir=DATA_DIR, file_name="noisy_kappa_val.npy")
label_val = Utility.load_np(data_dir=DATA_DIR, file_name="label_val.npy")

Ntrain = label_train.shape[0]*label_train.shape[1]
Nval = label_val.shape[0]*label_val.shape[1]

In [ ]:
# Reshape the data for CNN
X_train = noisy_kappa_train.reshape(Ntrain, *data_obj.shape)
X_val = noisy_kappa_val.reshape(Nval, *data_obj.shape)

# Here, we ignore the nuisance parameters and only keep the 2 cosmological parameters
y_train = label_train.reshape(Ntrain, 5)[:, :2]
y_val = label_val.reshape(Nval, 5)[:, :2]

In [ ]:
print(f'Shape of the split training data = {X_train.shape}')
print(f'Shape of the split validation data = {X_val.shape}')

print(f'Shape of the split training labels = {y_train.shape}')
print(f'Shape of the split validation labels = {y_val.shape}')

In [ ]:
# mask
Visualization.plot_mask(mask=data_obj.mask)

In [ ]:
# noiseless training convergence map
Visualization.plot_noiseless_training_convergence_map(kappa=data_obj.kappa)

In [ ]:
# noisy training convergence map
Visualization.plot_noisy_training_convergence_map(kappa=data_obj.kappa,
                                                  mask=data_obj.mask,
                                                  pixelsize_arcmin=data_obj.pixelsize_arcmin,
                                                  ng=data_obj.ng)

In [ ]:
Visualization.plot_cosmological_parameters_OmegaM_S8(label=data_obj.viz_label)

In [ ]:
Visualization.plot_baryonic_physics_parameters(label=data_obj.viz_label)

In [ ]:
Visualization.plot_photometric_redshift_uncertainty_parameters(label=data_obj.viz_label)

In [ ]:
# Define your path for saving the trained model
MODEL_SAVE_PATH = os.path.join(root_dir, "Phase1_starting_kit_CNN_MCMC_baseline.pth")

class Config:
    IMG_HEIGHT = data_obj.shape[0]
    IMG_WIDTH = data_obj.shape[1]
    
    # Parameters to predict (Omega_m, S_8)
    NUM_TARGETS = 2

    # Training hyperparameters
    BATCH_SIZE = 64
    EPOCHS = 15
    LEARNING_RATE = 2e-4
    WEIGHT_DECAY = 1e-4
    
    # Model hyperparameters
    DROPOUT1 = 0.2
    DROPOUT2 = 0.1
    
    DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
    MODEL_SAVE_PATH = MODEL_SAVE_PATH

In [ ]:
# Configuration for CNN-RF Ensemble
class EnsembleConfig:
    IMG_HEIGHT = data_obj.shape[0]
    IMG_WIDTH = data_obj.shape[1]
    
    # Parameters to predict (Omega_m, S_8)
    NUM_TARGETS = 2

    # Training hyperparameters for CNN
    BATCH_SIZE = 64
    EPOCHS = 15
    LEARNING_RATE = 2e-4
    WEIGHT_DECAY = 1e-4
    
    # Model hyperparameters
    FEATURE_DIM = 128  # Dimension of CNN features
    DROPOUT1 = 0.2
    DROPOUT2 = 0.1
    
    # Random Forest hyperparameters
    RF_PARAMS = {
        'n_estimators': 200,
        'max_depth': 20,
        'min_samples_split': 5,
        'min_samples_leaf': 2,
        'random_state': 42,
        'n_jobs': -1
    }
    
    DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
    CNN_MODEL_SAVE_PATH = os.path.join(root_dir, "Phase1_CNN_feature_extractor.pth")
    RF_MODEL_SAVE_PATH = os.path.join(root_dir, "Phase1_RF_model.pkl")

In [ ]:
# Initialize Ensemble Configuration and Model
ensemble_config = EnsembleConfig()
print(f"Using device: {ensemble_config.DEVICE}")
print(f"CNN Feature dimension: {ensemble_config.FEATURE_DIM}")
print(f"Random Forest params: {ensemble_config.RF_PARAMS}")

# Initialize CNN Feature Extractor with auxiliary prediction head
cnn_feature_model = CNN_FeatureExtractor_WithHead(
    height=ensemble_config.IMG_HEIGHT,
    width=ensemble_config.IMG_WIDTH,
    num_targets=ensemble_config.NUM_TARGETS,
    feature_dim=ensemble_config.FEATURE_DIM,
    dropout1=ensemble_config.DROPOUT1,
    dropout2=ensemble_config.DROPOUT2
).to(ensemble_config.DEVICE)

In [ ]:
# Train CNN-RF Ensemble

USE_PRETRAINED_ENSEMBLE = False
# USE_PRETRAINED_ENSEMBLE = True

if not USE_PRETRAINED_ENSEMBLE:
    print("=" * 60)
    print("STEP 1: Training CNN Feature Extractor")
    print("=" * 60)
    
    # Train the CNN feature extractor with auxiliary prediction head
    loss_fn = nn.MSELoss()
    optimizer = torch.optim.Adam(cnn_feature_model.parameters(),
                                lr=ensemble_config.LEARNING_RATE,
                                weight_decay=ensemble_config.WEIGHT_DECAY)
    scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=5)
    
    best_val_loss = float('inf')
    start_time = time.time()
    
    for epoch in range(ensemble_config.EPOCHS):
        train_loss = train_epoch_ensemble(cnn_feature_model, train_loader, loss_fn, optimizer, ensemble_config.DEVICE)
        val_loss = validate_epoch_ensemble(cnn_feature_model, val_loader, loss_fn, ensemble_config.DEVICE)
        
        scheduler.step(val_loss)
        print(f"Epoch {epoch+1}/{ensemble_config.EPOCHS} | Train Loss: {train_loss:.6f} | Val Loss: {val_loss:.6f}")
        
        # Save the best model
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            torch.save(cnn_feature_model.state_dict(), ensemble_config.CNN_MODEL_SAVE_PATH)
            print(f"  -> New best CNN model saved!")
    
    end_time = time.time()
    print(f"\nCNN training finished in {(end_time - start_time)/60:.2f} minutes.")
    
    # Load the best CNN model
    cnn_feature_model.load_state_dict(torch.load(ensemble_config.CNN_MODEL_SAVE_PATH, weights_only=True))
    
    print("\n" + "=" * 60)
    print("STEP 2: Training Random Forest on CNN Features")
    print("=" * 60)
    
    # Extract the feature extractor (without prediction head)
    feature_extractor = cnn_feature_model.get_feature_extractor()
    
    # Create ensemble with trained feature extractor
    ensemble_model = CNN_RF_Ensemble(feature_extractor, rf_params=ensemble_config.RF_PARAMS)
    
    # Train Random Forest on extracted features
    ensemble_model.fit_rf(train_loader)
    
    # Save Random Forest model
    import pickle
    with open(ensemble_config.RF_MODEL_SAVE_PATH, 'wb') as f:
        pickle.dump(ensemble_model.rf_model, f)
    print(f"Random Forest model saved to {ensemble_config.RF_MODEL_SAVE_PATH}")
    
    print("\n" + "=" * 60)
    print("Ensemble training complete!")
    print("=" * 60)
    
else:
    # Load pretrained models
    import pickle
    
    if os.path.exists(ensemble_config.CNN_MODEL_SAVE_PATH) and os.path.exists(ensemble_config.RF_MODEL_SAVE_PATH):
        print("Loading pretrained ensemble models...")
        
        # Load CNN feature extractor
        cnn_feature_model.load_state_dict(torch.load(ensemble_config.CNN_MODEL_SAVE_PATH, weights_only=True))
        feature_extractor = cnn_feature_model.get_feature_extractor()
        
        # Create ensemble and load RF model
        ensemble_model = CNN_RF_Ensemble(feature_extractor, rf_params=ensemble_config.RF_PARAMS)
        with open(ensemble_config.RF_MODEL_SAVE_PATH, 'rb') as f:
            ensemble_model.rf_model = pickle.load(f)
        
        print("Pretrained models loaded successfully!")
    else:
        warning_msg = "Pretrained ensemble models don't exist. Please train first."
        warnings.warn(warning_msg)

In [ ]:
# Validation predictions using CNN-RF Ensemble

print("Making predictions on validation set using CNN-RF Ensemble...")
y_pred_val_ensemble = ensemble_model.predict(val_loader)

# Inverse transform predictions back to original scale
y_pred_val_ensemble = label_scaler.inverse_transform(y_pred_val_ensemble)

print(f"Validation predictions shape: {y_pred_val_ensemble.shape}")

In [ ]:
# Comparison of CNN-RF Ensemble predictions vs validation labels

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Omega_m plot
axes[0].scatter(y_val[:,0], y_pred_val_ensemble[:,0], alpha=0.6, s=20)
axes[0].plot(sorted(y_val[:,0]), sorted(y_val[:,0]),
             color='grey', linestyle='dashed', linewidth=2, label='Perfect prediction')
axes[0].set_xlim(np.min(y_val[:,0]), np.max(y_val[:,0]))
axes[0].set_ylim(0, 0.7)
axes[0].set_xlabel('Ground Truth', fontsize=12)
axes[0].set_ylabel('Prediction', fontsize=12)
axes[0].set_title(r'$\Omega_m$ (CNN-RF Ensemble)', fontsize=14)
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# S_8 plot
axes[1].scatter(y_val[:,1], y_pred_val_ensemble[:,1], alpha=0.6, s=20)
axes[1].plot(sorted(y_val[:,1]), sorted(y_val[:,1]),
             color='grey', linestyle='dashed', linewidth=2, label='Perfect prediction')
axes[1].set_xlim(np.min(y_val[:,1]), np.max(y_val[:,1]))
axes[1].set_ylim(0.65, 1)
axes[1].set_xlabel('Ground Truth', fontsize=12)
axes[1].set_ylabel('Prediction', fontsize=12)
axes[1].set_title(r'$S_8$ (CNN-RF Ensemble)', fontsize=14)
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Calculate and display metrics
from sklearn.metrics import mean_squared_error, r2_score

mse_omega_m = mean_squared_error(y_val[:,0], y_pred_val_ensemble[:,0])
mse_s8 = mean_squared_error(y_val[:,1], y_pred_val_ensemble[:,1])
r2_omega_m = r2_score(y_val[:,0], y_pred_val_ensemble[:,0])
r2_s8 = r2_score(y_val[:,1], y_pred_val_ensemble[:,1])

print(f"\nValidation Metrics (CNN-RF Ensemble):")
print(f"  Omega_m - MSE: {mse_omega_m:.6f}, R²: {r2_omega_m:.4f}")
print(f"  S_8     - MSE: {mse_s8:.6f}, R²: {r2_s8:.4f}")

In [ ]:
# Prepare summary statistics for MCMC using ensemble predictions

# Group validation indices by cosmology (same as before)
cosmology = data_obj.label[:,0,:2]
row_to_i = {tuple(cosmology[i]): i for i in range(Ncosmo)}
index_lists = [[] for _ in range(cosmology.shape[0])]

for idx in range(len(y_val)):
    row_tuple = tuple(y_val[idx])
    i = row_to_i[row_tuple]
    index_lists[i].append(idx)

val_cosmology_idx_ensemble = [np.array(lst) for lst in index_lists]

# Summary statistics using ensemble predictions
d_vector_ensemble = []
n_d = 2

for i in range(Ncosmo):
    d_i = np.zeros((len(val_cosmology_idx_ensemble[i]), n_d))
    for j, idx in enumerate(val_cosmology_idx_ensemble[i]):
        d_i[j] = y_pred_val_ensemble[idx]
    d_vector_ensemble.append(d_i)

# Mean summary statistics
mean_d_vector_ensemble = []
for i in range(Ncosmo):
    mean_d_vector_ensemble.append(np.mean(d_vector_ensemble[i], 0))
mean_d_vector_ensemble = np.array(mean_d_vector_ensemble)

# Covariance matrix
delta_ensemble = []
for i in range(Ncosmo):
    delta_ensemble.append((d_vector_ensemble[i] - mean_d_vector_ensemble[i].reshape(1, n_d)))

cov_d_vector_ensemble = [(delta_ensemble[i].T @ delta_ensemble[i] / (len(delta_ensemble[i])-n_d-2))[None] 
                          for i in range(Ncosmo)]
cov_d_vector_ensemble = np.concatenate(cov_d_vector_ensemble, 0)

print("Summary statistics computed for ensemble predictions")

In [ ]:
# Interpolators for MCMC with ensemble predictions

from scipy.interpolate import LinearNDInterpolator

mean_d_vector_interp_ensemble = LinearNDInterpolator(cosmology, mean_d_vector_ensemble, fill_value=np.nan)
cov_d_vector_interp_ensemble = LinearNDInterpolator(cosmology, cov_d_vector_ensemble, fill_value=np.nan)
logprior_interp_ensemble = LinearNDInterpolator(cosmology, np.zeros((Ncosmo, 1)), fill_value=-np.inf)

def log_prior_ensemble(x):
    logprior = logprior_interp_ensemble(x).flatten()
    return logprior

def loglike_ensemble(x, d):
    mean = mean_d_vector_interp_ensemble(x)
    cov = cov_d_vector_interp_ensemble(x)
    delta = d - mean
    
    inv_cov = np.linalg.inv(cov)
    cov_det = np.linalg.slogdet(cov)[1]
    
    return -0.5 * cov_det - 0.5 * np.einsum("ni,nij,nj->n", delta, inv_cov, delta)

def logp_posterior_ensemble(x, d):
    logp = log_prior_ensemble(x)
    select = np.isfinite(logp)
    if np.sum(select) > 0:
        logp[select] = logp[select] + loglike_ensemble(x[select], d[select])
    return logp

print("MCMC interpolators created for ensemble")

In [ ]:
# MCMC sampling for validation set with ensemble predictions

Nstep_ensemble = 10000
sigma_ensemble = 0.06

current_ensemble = cosmology[np.random.choice(Ncosmo, size=Nval)]
curr_logprob_ensemble = logp_posterior_ensemble(current_ensemble, y_pred_val_ensemble)

states_ensemble = []
total_acc_ensemble = np.zeros(len(current_ensemble))

print("Running MCMC for validation set with ensemble predictions...")
t = time.time()

for i in range(Nstep_ensemble):
    proposal = current_ensemble + np.random.randn(*current_ensemble.shape) * sigma_ensemble
    proposal_logprob = logp_posterior_ensemble(proposal, y_pred_val_ensemble)
    
    acc_logprob = proposal_logprob - curr_logprob_ensemble
    acc_logprob[acc_logprob > 0] = 0
    
    acc_prob = np.exp(acc_logprob)
    acc = np.random.uniform(size=len(current_ensemble)) < acc_prob
    
    total_acc_ensemble += acc_prob
    current_ensemble[acc] = proposal[acc]
    curr_logprob_ensemble[acc] = proposal_logprob[acc]
    
    states_ensemble.append(np.copy(current_ensemble)[None])
    
    if i % (0.1*Nstep_ensemble) == 0.1*Nstep_ensemble-1:
        print(
            'Step:', len(states_ensemble),
            'Time:', f'{time.time() - t:.2f}s',
            'Min acceptance rate:', f'{np.min(total_acc_ensemble / (i + 1)):.4f}',
            'Mean acceptance rate:', f'{np.mean(total_acc_ensemble / (i + 1)):.4f}'
        )
        t = time.time()

# Remove burn-in
states_ensemble = np.concatenate(states_ensemble[int(0.2*Nstep_ensemble):], 0)

# Mean and std of samples
mean_val_ensemble = np.mean(states_ensemble, 0)
errorbar_val_ensemble = np.std(states_ensemble, 0)

print(f"\nMCMC complete. Mean error bars: {np.mean(errorbar_val_ensemble, 0)}")

In [ ]:
# Visualization of MCMC results with error bars (Ensemble)

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Omega_m plot with error bars
axes[0].errorbar(y_val[:,0], mean_val_ensemble[:,0], yerr=errorbar_val_ensemble[:,0],
                 fmt='o', capsize=3, capthick=1, ecolor='grey', alpha=0.6, markersize=4)
axes[0].plot(sorted(y_val[:,0]), sorted(y_val[:,0]),
             color='grey', linestyle='dashed', linewidth=2, label='Perfect prediction')
axes[0].set_xlim(np.min(y_val[:,0]), np.max(y_val[:,0]))
axes[0].set_ylim(0, 0.7)
axes[0].set_xlabel('Ground Truth', fontsize=12)
axes[0].set_ylabel('Prediction', fontsize=12)
axes[0].set_title(r'$\Omega_m$ (CNN-RF Ensemble + MCMC)', fontsize=14)
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# S_8 plot with error bars
axes[1].errorbar(y_val[:,1], mean_val_ensemble[:,1], yerr=errorbar_val_ensemble[:,1],
                 fmt='o', capsize=3, capthick=1, ecolor='grey', alpha=0.6, markersize=4)
axes[1].plot(sorted(y_val[:,1]), sorted(y_val[:,1]),
             color='grey', linestyle='dashed', linewidth=2, label='Perfect prediction')
axes[1].set_xlim(np.min(y_val[:,1]), np.max(y_val[:,1]))
axes[1].set_ylim(0.65, 1)
axes[1].set_xlabel('Ground Truth', fontsize=12)
axes[1].set_ylabel('Prediction', fontsize=12)
axes[1].set_title(r'$S_8$ (CNN-RF Ensemble + MCMC)', fontsize=14)
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Calculate validation score for ensemble

validation_score_ensemble = Score._score_phase1(
    true_cosmo=y_val,
    infer_cosmo=mean_val_ensemble,
    errorbar=errorbar_val_ensemble
)

print('=' * 60)
print('VALIDATION RESULTS (CNN-RF Ensemble)')
print('=' * 60)
print(f'Average score: {np.mean(validation_score_ensemble):.2f}')
print(f'Average error bar (Omega_m): {np.mean(errorbar_val_ensemble[:, 0]):.6f}')
print(f'Average error bar (S_8): {np.mean(errorbar_val_ensemble[:, 1]):.6f}')
print('=' * 60)

In [ ]:
# Test set predictions using CNN-RF Ensemble

print("Making predictions on test set using CNN-RF Ensemble...")
y_pred_test_ensemble = ensemble_model.predict(test_loader)

# Inverse transform predictions back to original scale
y_pred_test_ensemble = label_scaler.inverse_transform(y_pred_test_ensemble)

print(f"Test predictions shape: {y_pred_test_ensemble.shape}")
print(f"Sample predictions (first 5):\n{y_pred_test_ensemble[:5]}")

In [ ]:
# MCMC sampling for test set with ensemble predictions

Nstep_test = 10000
sigma_test = 0.06

current_test = cosmology[np.random.choice(Ncosmo, size=data_obj.Ntest)]
curr_logprob_test = logp_posterior_ensemble(current_test, y_pred_test_ensemble)

states_test = []
total_acc_test = np.zeros(len(current_test))

print("Running MCMC for test set with ensemble predictions...")
t = time.time()

for i in range(Nstep_test):
    proposal = current_test + np.random.randn(*current_test.shape) * sigma_test
    proposal_logprob = logp_posterior_ensemble(proposal, y_pred_test_ensemble)
    
    acc_logprob = proposal_logprob - curr_logprob_test
    acc_logprob[acc_logprob > 0] = 0
    
    acc_prob = np.exp(acc_logprob)
    acc = np.random.uniform(size=len(current_test)) < acc_prob
    
    total_acc_test += acc_prob
    current_test[acc] = proposal[acc]
    curr_logprob_test[acc] = proposal_logprob[acc]
    
    states_test.append(np.copy(current_test)[None])
    
    if i % (0.1*Nstep_test) == 0.1*Nstep_test-1:
        print(
            'Step:', len(states_test),
            'Time:', f'{time.time() - t:.2f}s',
            'Min acceptance rate:', f'{np.min(total_acc_test / (i + 1)):.4f}',
            'Mean acceptance rate:', f'{np.mean(total_acc_test / (i + 1)):.4f}'
        )
        t = time.time()

# Remove burn-in
states_test = np.concatenate(states_test[int(0.2*Nstep_test):], 0)

# Mean and std of samples
mean_test_ensemble = np.mean(states_test, 0)
errorbar_test_ensemble = np.std(states_test, 0)

print(f"\nMCMC complete. Mean error bars: {np.mean(errorbar_test_ensemble, 0)}")
print(f"Sample means (first 5):\n{mean_test_ensemble[:5]}")
print(f"Sample error bars (first 5):\n{errorbar_test_ensemble[:5]}")

In [ ]:
# Save submission file for CNN-RF Ensemble

data_ensemble = {
    "means": mean_test_ensemble.tolist(),
    "errorbars": errorbar_test_ensemble.tolist()
}

the_date = datetime.datetime.now().strftime("%y-%m-%d-%H-%M")
zip_file_name_ensemble = 'Submission_Ensemble_' + the_date + '.zip'

zip_file_ensemble = Utility.save_json_zip(
    submission_dir="submissions",
    json_file_name="result.json",
    zip_file_name=zip_file_name_ensemble,
    data=data_ensemble
)

print("=" * 60)
print("SUBMISSION FILE CREATED (CNN-RF Ensemble)")
print("=" * 60)
print(f"Submission ZIP saved at: {zip_file_ensemble}")
print("=" * 60)

In [ ]:
# Summary: CNN-RF Ensemble Pipeline

print("=" * 80)
print("CNN-RANDOM FOREST ENSEMBLE PIPELINE SUMMARY")
print("=" * 80)
print("\nArchitecture:")
print("  1. CNN Feature Extractor:")
print("     - 4 convolutional blocks with batch normalization")
print("     - Extracts 128-dimensional features from convergence maps")
print("     - Trained with auxiliary prediction head using MSE loss")
print("\n  2. Random Forest Regressor:")
print(f"     - n_estimators: {ensemble_config.RF_PARAMS['n_estimators']}")
print(f"     - max_depth: {ensemble_config.RF_PARAMS['max_depth']}")
print(f"     - Trained on CNN-extracted features")
print("\n  3. MCMC Uncertainty Quantification:")
print("     - Metropolis-Hastings sampling")
print("     - 10,000 steps with 20% burn-in")
print("\nAdvantages of CNN-RF Ensemble:")
print("  ✓ CNN captures spatial features and patterns")
print("  ✓ Random Forest provides non-linear combination of features")
print("  ✓ Ensemble reduces overfitting compared to pure neural network")
print("  ✓ Random Forest is less prone to gradient issues")
print("  ✓ Better generalization on unseen data")
print("=" * 80)

In [ ]:
# Simple CNN architecture for parameter estimation

class Simple_CNN(nn.Module):
    def __init__(self, height, width, num_targets, dropout1=0.2, dropout2=0.1):
        super(Simple_CNN, self).__init__()
        self.conv_stack = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=5, stride=2, padding=2),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )

        self._feature_size = self._get_conv_output_size(height, width)
        
        self.fc_stack = nn.Sequential(
            nn.Flatten(),
            nn.Linear(self._feature_size, 512),
            nn.ReLU(),
            nn.Dropout(dropout1),
            nn.Linear(512, 128),
            nn.ReLU(),
            nn.Dropout(dropout2),
            nn.Linear(128, num_targets)
        )

    def _get_conv_output_size(self, height, width):
        dummy_input = torch.zeros(1, 1, height, width)
        output = self.conv_stack(dummy_input)
        return int(np.prod(output.size()))

    def forward(self, x):
        x = self.conv_stack(x)
        x = self.fc_stack(x)
        return x

In [ ]:
# CNN Feature Extractor - modified to output features instead of predictions

class CNN_FeatureExtractor(nn.Module):
    """
    CNN that extracts features from the penultimate layer
    instead of making direct predictions.
    """
    def __init__(self, height, width, feature_dim=128, dropout1=0.2, dropout2=0.1):
        super(CNN_FeatureExtractor, self).__init__()
        self.conv_stack = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=5, stride=2, padding=2),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )

        self._feature_size = self._get_conv_output_size(height, width)
        
        # Feature extraction layers (no final prediction layer)
        self.feature_extractor = nn.Sequential(
            nn.Flatten(),
            nn.Linear(self._feature_size, 512),
            nn.ReLU(),
            nn.Dropout(dropout1),
            nn.Linear(512, feature_dim),
            nn.ReLU(),
            nn.Dropout(dropout2)
        )

    def _get_conv_output_size(self, height, width):
        dummy_input = torch.zeros(1, 1, height, width)
        output = self.conv_stack(dummy_input)
        return int(np.prod(output.size()))

    def forward(self, x):
        x = self.conv_stack(x)
        x = self.feature_extractor(x)
        return x


# CNN-Random Forest Ensemble
class CNN_RF_Ensemble:
    """
    Ensemble model combining CNN feature extraction with Random Forest regression.
    """
    def __init__(self, cnn_model, rf_params=None):
        """
        Parameters
        ----------
        cnn_model : CNN_FeatureExtractor
            Trained CNN feature extractor
        rf_params : dict, optional
            Parameters for RandomForestRegressor
        """
        self.cnn_model = cnn_model
        self.device = next(cnn_model.parameters()).device
        
        if rf_params is None:
            rf_params = {
                'n_estimators': 200,
                'max_depth': 20,
                'min_samples_split': 5,
                'min_samples_leaf': 2,
                'random_state': 42,
                'n_jobs': -1
            }
        self.rf_model = RandomForestRegressor(**rf_params)
        
    def extract_features(self, dataloader):
        """Extract features from images using CNN"""
        self.cnn_model.eval()
        features_list = []
        labels_list = []
        
        with torch.no_grad():
            for batch in tqdm(dataloader, desc="Extracting features"):
                if len(batch) == 2:
                    X, y = batch
                    X = X.to(self.device)
                    features = self.cnn_model(X)
                    features_list.append(features.cpu().numpy())
                    labels_list.append(y.numpy())
                else:
                    X = batch
                    X = X.to(self.device)
                    features = self.cnn_model(X)
                    features_list.append(features.cpu().numpy())
        
        features = np.concatenate(features_list, axis=0)
        if labels_list:
            labels = np.concatenate(labels_list, axis=0)
            return features, labels
        return features
    
    def fit_rf(self, train_loader):
        """Train Random Forest on CNN features"""
        print("Extracting training features...")
        X_train_features, y_train = self.extract_features(train_loader)
        
        print(f"Training Random Forest on {X_train_features.shape[0]} samples...")
        self.rf_model.fit(X_train_features, y_train)
        print("Random Forest training complete!")
        
    def predict(self, test_loader):
        """Make predictions using CNN features + Random Forest"""
        print("Extracting test features...")
        X_test_features = self.extract_features(test_loader)
        
        print("Making predictions with Random Forest...")
        predictions = self.rf_model.predict(X_test_features)
        return predictions

# CNN-Random Forest Ensemble Approach

## Overview
This section implements an **ensemble learning approach** that combines the strengths of Convolutional Neural Networks (CNNs) and Random Forests:

### Architecture
1. **CNN Feature Extractor**: 
   - Processes convergence maps through convolutional layers
   - Extracts high-level spatial features (128-dimensional vectors)
   - Trained with an auxiliary prediction head using supervised learning

2. **Random Forest Regressor**:
   - Takes CNN-extracted features as input
   - Learns non-linear relationships between features and cosmological parameters
   - Provides robust predictions with built-in feature importance

3. **MCMC Uncertainty Quantification**:
   - Uses Metropolis-Hastings sampling to estimate posterior distributions
   - Provides confidence intervals for predictions

### Advantages
- **Better Feature Learning**: CNN learns spatial patterns in convergence maps
- **Improved Generalization**: Random Forest reduces overfitting
- **Robustness**: Ensemble approach is more stable than single model
- **Interpretability**: Random Forest provides feature importance insights

### Workflow
1. Train CNN feature extractor with auxiliary head
2. Extract features from training data
3. Train Random Forest on extracted features
4. Make predictions by passing data through CNN → RF pipeline
5. Use MCMC for uncertainty quantification

In [ ]:
def train_epoch(model, dataloader, loss_fn, optimizer, device):
    model.train()
    total_loss = 0
    pbar = tqdm(dataloader, total=len(dataloader), desc="Training")
    for X, y in pbar:
        X, y = X.to(device), y.to(device)

        # Forward pass
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
    
    return total_loss / len(dataloader)


def validate_epoch(model, dataloader, loss_fn, device):
    model.eval()
    total_loss = 0
    pbar = tqdm(dataloader, total=len(dataloader), desc="Validating")
    with torch.no_grad():
        for X, y in pbar:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            total_loss += loss_fn(pred, y).item()
            
    return total_loss / len(dataloader)

In [ ]:
# Training function for CNN Feature Extractor (with auxiliary prediction head for training)

class CNN_FeatureExtractor_WithHead(nn.Module):
    """
    CNN Feature Extractor with an auxiliary prediction head for training.
    The prediction head is used during training but discarded during feature extraction.
    """
    def __init__(self, height, width, num_targets, feature_dim=128, dropout1=0.2, dropout2=0.1):
        super(CNN_FeatureExtractor_WithHead, self).__init__()
        self.feature_extractor = CNN_FeatureExtractor(height, width, feature_dim, dropout1, dropout2)
        self.prediction_head = nn.Linear(feature_dim, num_targets)
        
    def forward(self, x, return_features=False):
        features = self.feature_extractor(x)
        if return_features:
            return features
        predictions = self.prediction_head(features)
        return predictions
    
    def get_feature_extractor(self):
        """Return just the feature extractor without the prediction head"""
        return self.feature_extractor


def train_epoch_ensemble(model, dataloader, loss_fn, optimizer, device):
    """Training epoch for CNN with auxiliary head"""
    model.train()
    total_loss = 0
    pbar = tqdm(dataloader, total=len(dataloader), desc="Training CNN Feature Extractor")
    for X, y in pbar:
        X, y = X.to(device), y.to(device)

        # Forward pass
        pred = model(X, return_features=False)
        loss = loss_fn(pred, y)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
        pbar.set_postfix({'loss': f'{loss.item():.6f}'})
    
    return total_loss / len(dataloader)


def validate_epoch_ensemble(model, dataloader, loss_fn, device):
    """Validation epoch for CNN with auxiliary head"""
    model.eval()
    total_loss = 0
    pbar = tqdm(dataloader, total=len(dataloader), desc="Validating CNN Feature Extractor")
    with torch.no_grad():
        for X, y in pbar:
            X, y = X.to(device), y.to(device)
            pred = model(X, return_features=False)
            loss = loss_fn(pred, y)
            total_loss += loss.item()
            pbar.set_postfix({'loss': f'{loss.item():.6f}'})
            
    return total_loss / len(dataloader)

In [ ]:
def objective(trial):
    # Suggest hyperparameters
    batch_size = trial.suggest_categorical('batch_size', [32, 64, 128])
    learning_rate = trial.suggest_float('learning_rate', 1e-5, 1e-3, log=True)
    weight_decay = trial.suggest_float('weight_decay', 1e-5, 1e-3, log=True)
    dropout1 = trial.suggest_float('dropout1', 0.0, 0.5)
    dropout2 = trial.suggest_float('dropout2', 0.0, 0.5)
    epochs = trial.suggest_int('epochs', 3, 10)
    
    # Create model
    model = Simple_CNN(config.IMG_HEIGHT, config.IMG_WIDTH, config.NUM_TARGETS, dropout1, dropout2).to(config.DEVICE)
    
    # Create dataloaders with suggested batch_size
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
    
    # Loss and optimizer
    loss_fn = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
    
    # Train for suggested epochs
    best_val_loss = float('inf')
    for epoch in range(epochs):
        train_loss = train_epoch(model, train_loader, loss_fn, optimizer, config.DEVICE)
        val_loss = validate_epoch(model, val_loader, loss_fn, config.DEVICE)
        if val_loss < best_val_loss:
            best_val_loss = val_loss
    
    return best_val_loss

In [ ]:
# Run Optuna hyperparameter optimization
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=20)

best_params = study.best_params
print("Best hyperparameters:", best_params)

In [ ]:
class CosmologyDataset(Dataset):
    """
    Custom PyTorch Dataset
    """
    
    def __init__(self, data, labels=None,
                 transform=None,
                 label_transform=None):
        self.data = data
        self.labels = labels
        self.transform = transform
        self.label_transform = label_transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        image = self.data[idx].astype(np.float32)   # Convert from float16 to float32
        if self.transform:
            image = self.transform(image) 
        if self.labels is not None:
            label = self.labels[idx].astype(np.float32)
            label = torch.from_numpy(label)
            if self.label_transform:
                label = self.label_transform(label)
            return image, label
        else:
            return image

In [ ]:
# Compute the means and stds of the training images (for standardizing the data)

means = np.mean(X_train, dtype=np.float32)
stds = np.std(X_train, dtype=np.float32)

In [ ]:
# Image standardization
from torchvision import transforms
transform = transforms.Compose([
    transforms.ToTensor(),     
    transforms.Normalize(mean=[means], std=[stds]),   
])
print(f"Image stats (from train set): Mean={means}, Std={stds}")

# Label standardization
label_scaler = StandardScaler()
y_train_scaled = label_scaler.fit_transform(y_train)
y_val_scaled = label_scaler.transform(y_val)
print(f"Label stats (from train set): Mean={label_scaler.mean_}, Std={np.sqrt(label_scaler.var_)}")

In [ ]:
# Load the configuration
config = Config()
print(f"Using device: {config.DEVICE}")

# Create Datasets and DataLoaders
train_dataset = CosmologyDataset(
    data=X_train, 
    labels=y_train_scaled,
    transform=transform
)
val_dataset = CosmologyDataset(
    data=X_val, 
    labels=y_val_scaled,
    transform=transform
)

train_loader = DataLoader(train_dataset, batch_size=config.BATCH_SIZE, shuffle=True) 
val_loader = DataLoader(val_dataset, batch_size=config.BATCH_SIZE, shuffle=False)

In [ ]:
# Initialize the CNN model
model = Simple_CNN(config.IMG_HEIGHT,
                    config.IMG_WIDTH,
                    config.NUM_TARGETS,
                    config.DROPOUT1,
                    config.DROPOUT2).to(config.DEVICE)

In [ ]:
USE_PRETRAINED_MODEL = False
# USE_PRETRAINED_MODEL = True

In [ ]:
# Update config with best hyperparameters
config.BATCH_SIZE = best_params['batch_size']
config.LEARNING_RATE = best_params['learning_rate']
config.WEIGHT_DECAY = best_params['weight_decay']
config.DROPOUT1 = best_params['dropout1']
config.DROPOUT2 = best_params['dropout2']
config.EPOCHS = best_params['epochs']

# Recreate model with best dropouts
model = Simple_CNN(config.IMG_HEIGHT, config.IMG_WIDTH, config.NUM_TARGETS, config.DROPOUT1, config.DROPOUT2).to(config.DEVICE)

# Recreate dataloaders with best batch_size
train_loader = DataLoader(train_dataset, batch_size=config.BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=config.BATCH_SIZE, shuffle=False)

In [ ]:
if not USE_PRETRAINED_MODEL:  
    # Train the model
    loss_fn = nn.MSELoss() 
    optimizer = torch.optim.Adam(model.parameters(),
                                lr=config.LEARNING_RATE,
                                weight_decay=config.WEIGHT_DECAY)
    scheduler = ReduceLROnPlateau(optimizer,
                                  mode='min',
                                  factor=0.5,
                                  patience=5)
    # Training Loop
    best_val_loss = float('inf')
    start_time = time.time()
    for epoch in range(config.EPOCHS):
        train_loss = train_epoch(model, train_loader, loss_fn, optimizer, config.DEVICE)
        val_loss = validate_epoch(model, val_loader, loss_fn, config.DEVICE)
    
        scheduler.step(val_loss)    
        print(f"Epoch {epoch+1}/{config.EPOCHS} | Train Loss: {train_loss:.6f} | Val Loss: {val_loss:.6f}")
        
        # Save the best model based on validation loss
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            torch.save(model.state_dict(), config.MODEL_SAVE_PATH)
            print(f"  -> New best model saved to {config.MODEL_SAVE_PATH}")

    end_time = time.time()
    print(f"\nTraining finished in {(end_time - start_time)/60:.2f} minutes.")
    
    model.load_state_dict(torch.load(config.MODEL_SAVE_PATH, weights_only=True)) # Directly load the best model

else:
    # Check if the pretrained model exists
    if os.path.exists(config.MODEL_SAVE_PATH):
        # If the pretrained model exists, load the model
        model.load_state_dict(torch.load(config.MODEL_SAVE_PATH, weights_only=True))

    else:
        # If the pretrained model doesn't exist, show the warning message
        warning_msg = f"The path of pretrained model doesn't exist"
        warnings.warn(warning_msg)

In [ ]:
model.eval()
y_pred_list = []   
pbar = tqdm(val_loader, total=len(val_loader), desc="Validating")
with torch.no_grad():
    for X, _ in pbar:
        X = X.to(config.DEVICE)
        y_pred = model(X)        
        y_pred = label_scaler.inverse_transform(y_pred.cpu().numpy())
        y_pred_list.append(y_pred) 

y_pred_val = np.concatenate(y_pred_list, axis=0)

In [ ]:
# Comparison of the CNN predictions and the validation labels

plt.scatter(y_val[:,0], y_pred_val[:,0])
plt.plot(sorted(y_val[:,0]), sorted(y_val[:,0]),
         color = 'grey', linestyle='dashed')
plt.xlim(np.min(y_val[:,0]), np.max(y_val[:,0]))
plt.ylim(0, 0.7)
plt.xlabel('Ground Truth')
plt.ylabel('Prediction')
plt.title(r'$\Omega_m$')
plt.show()

plt.scatter(y_val[:,1], y_pred_val[:,1])
plt.plot(sorted(y_val[:,1]), sorted(y_val[:,1]),
         color = 'grey', linestyle='dashed')
plt.xlim(np.min(y_val[:,1]), np.max(y_val[:,1]))
plt.ylim(0.65, 1)
plt.xlabel('Ground Truth')
plt.ylabel('Prediction')
plt.title(r'$S_8$')
plt.show()

In [ ]:
# There are Ncosmo distinct cosmologies in the labels.
# Here we create a list that groups the indices of the validation instances with the same cosmological parameters

cosmology = data_obj.label[:,0,:2]   # shape = (Ncosmo, 2)

row_to_i = {tuple(cosmology[i]): i for i in range(Ncosmo)}
index_lists = [[] for _ in range(cosmology.shape[0])]

# Loop over each row in 'y_val' with shape = (Nval, 2)
for idx in range(len(y_val)):
    row_tuple = tuple(y_val[idx])
    i = row_to_i[row_tuple]
    index_lists[i].append(idx)

# val_cosmology_idx[i] = the indices idx of the validation examples with labels = cosmology[i]
val_cosmology_idx = [np.array(lst) for lst in index_lists]  

In [ ]:
# The summary statistics of all realizations for all cosmologies in the validation set
d_vector = []  
n_d = 2   # Number of summary statistics for each map
for i in range(Ncosmo):
    d_i =  np.zeros((len(val_cosmology_idx[i]), n_d))  
    for j, idx in enumerate(val_cosmology_idx[i]):
        d_i[j] = y_pred_val[idx]

    d_vector.append(d_i)

In [ ]:
# mean summary statistics (average over all realizations)
mean_d_vector = []
for i in range(Ncosmo):
    mean_d_vector.append(np.mean(d_vector[i], 0))
mean_d_vector = np.array(mean_d_vector)   

# covariance matrix
delta = []
for i in range(Ncosmo):
    delta.append((d_vector[i] - mean_d_vector[i].reshape(1, n_d))) 

cov_d_vector = [(delta[i].T @ delta[i] / (len(delta[i])-n_d-2))[None] for i in range(Ncosmo)]     
cov_d_vector = np.concatenate(cov_d_vector, 0) 

In [ ]:
from scipy.interpolate import LinearNDInterpolator
mean_d_vector_interp = LinearNDInterpolator(cosmology, mean_d_vector, fill_value=np.nan)
cov_d_vector_interp = LinearNDInterpolator(cosmology, cov_d_vector, fill_value=np.nan)

In [ ]:
logprior_interp = LinearNDInterpolator(cosmology, np.zeros((Ncosmo, 1)), fill_value=-np.inf)

# Note that the training data are not uniformly sampled, which introduces a prior distribution. Here we ignore that prior for simplicity.
# Also note that this prior would introduce bias for cosmologies at the boundary of the prior
def log_prior(x):
    logprior = logprior_interp(x).flatten()  # shape = (Ntest, ) 
    return logprior

# Gaussian likelihood with interpolated mean and covariance matrix
def loglike(x, d):
    mean = mean_d_vector_interp(x) 
    cov = cov_d_vector_interp(x)   
    delta = d - mean               
    
    inv_cov = np.linalg.inv(cov)
    cov_det = np.linalg.slogdet(cov)[1]
    
    return -0.5 * cov_det - 0.5 * np.einsum("ni,nij,nj->n", delta, inv_cov, delta)

def logp_posterior(x, d):
    logp = log_prior(x)
    select = np.isfinite(logp)
    if np.sum(select) > 0:
        logp[select] = logp[select] + loglike(x[select], d[select])
    return logp

In [ ]:
# MCMC sampling to explore the posterior distribution

Nstep = 10000  # Number of MCMC steps (iterations)
sigma = 0.06   # Proposal standard deviation; should be tuned per method or parameter scale

# Randomly select initial points from the `cosmology` array for each test case
# Assumes `cosmology` has shape (Ncosmo, ndim) and `Ntest` is the number of independent chains/samples
current = cosmology[np.random.choice(Ncosmo, size=Nval)]

# Compute log-posterior at the initial points
curr_logprob = logp_posterior(current, y_pred_val)

# List to store sampled states (for all chains)
states = []

# Track total acceptance probabilities to compute acceptance rates
total_acc = np.zeros(len(current))

t = time.time()  # Track time for performance reporting

# MCMC loop
for i in range(Nstep):

    # Generate proposals by adding Gaussian noise to current state
    proposal = current + np.random.randn(*current.shape) * sigma    

    # Compute log-posterior at the proposed points
    proposal_logprob = logp_posterior(proposal, y_pred_val)

    # Compute log acceptance ratio (Metropolis-Hastings)
    acc_logprob = proposal_logprob - curr_logprob
    acc_logprob[acc_logprob > 0] = 0  # Cap at 0 to avoid exp overflow (acceptance prob ≤ 1)

    # Convert to acceptance probabilities
    acc_prob = np.exp(acc_logprob)

    # Decide whether to accept each proposal
    acc = np.random.uniform(size=len(current)) < acc_prob

    # Track acceptance probabilities (not binary outcomes)
    total_acc += acc_prob

    # Update states and log-probs where proposals are accepted
    current[acc] = proposal[acc]
    curr_logprob[acc] = proposal_logprob[acc]

    # Save a copy of the current state
    states.append(np.copy(current)[None])

    # Periodically print progress and acceptance rates
    if i % (0.1*Nstep) == 0.1*Nstep-1:
        print(
            'step:', len(states),
            'Time:', time.time() - t,
            'Min acceptance rate:', np.min(total_acc / (i + 1)),
            'Mean acceptance rate:', np.mean(total_acc / (i + 1))
        )
        t = time.time()  # Reset timer for next print interval

In [ ]:
# remove burn-in
states = np.concatenate(states[int(0.2*Nstep):], 0)

# mean and std of samples
mean_val = np.mean(states, 0)
errorbar_val = np.std(states, 0)

In [ ]:
# Comparison of the means & standard deviations of the posterior distributions and the validation labels

plt.errorbar(y_val[:,0], mean_val[:,0], yerr=errorbar_val[:,0], 
             fmt='o', capsize=3, capthick=1, ecolor='grey')
plt.plot(sorted(y_val[:,0]), sorted(y_val[:,0]),
         color = 'grey', linestyle='dashed')
plt.xlim(np.min(y_val[:,0]), np.max(y_val[:,0]))
plt.ylim(0, 0.7)
plt.xlabel('Ground Truth')
plt.ylabel('Prediction')
plt.title(r'$\Omega_m$')
plt.show()

plt.errorbar(y_val[:,1], mean_val[:,1], yerr=errorbar_val[:,1], 
             fmt='o', capsize=3, capthick=1, ecolor='grey')
plt.plot(sorted(y_val[:,1]), sorted(y_val[:,1]),
         color = 'grey', linestyle='dashed')
plt.xlim(np.min(y_val[:,1]), np.max(y_val[:,1]))
plt.ylim(0.65, 1)
plt.xlabel('Ground Truth')
plt.ylabel('Prediction')
plt.title(r'$S_8$')
plt.show()

In [ ]:
validation_score = Score._score_phase1(
    true_cosmo=y_val,
    infer_cosmo=mean_val,
    errorbar=errorbar_val
)
print('averaged score:', np.mean(validation_score))
print('averaged error bar:', np.mean(errorbar_val, 0))

In [ ]:
test_dataset = CosmologyDataset(
    data=data_obj.kappa_test, 
    transform=transform
)

test_loader = DataLoader(test_dataset, batch_size=config.BATCH_SIZE, shuffle=False)

In [ ]:
model.eval()
y_pred_list = []   
pbar = tqdm(test_loader, total=len(test_loader), desc="Inference on the test set")
with torch.no_grad():
    for X in pbar:
        X = X.to(config.DEVICE)
        y_pred = model(X)        
        y_pred = label_scaler.inverse_transform(y_pred.cpu().numpy())
        y_pred_list.append(y_pred) 

y_pred_test = np.concatenate(y_pred_list, axis=0)

In [ ]:
# MCMC sampling to explore the posterior distribution

Nstep = 10000  # Number of MCMC steps (iterations)
sigma = 0.06   # Proposal standard deviation; should be tuned per method or parameter scale

# Randomly select initial points from the `cosmology` array for each test case
# Assumes `cosmology` has shape (Ncosmo, ndim) and `Ntest` is the number of independent chains/samples
current = cosmology[np.random.choice(Ncosmo, size=data_obj.Ntest)]

# Compute log-posterior at the initial points
curr_logprob = logp_posterior(current, y_pred_test)

# List to store sampled states (for all chains)
states = []

# Track total acceptance probabilities to compute acceptance rates
total_acc = np.zeros(len(current))

t = time.time()  # Track time for performance reporting

# MCMC loop
for i in range(Nstep):

    # Generate proposals by adding Gaussian noise to current state
    proposal = current + np.random.randn(*current.shape) * sigma    

    # Compute log-posterior at the proposed points
    proposal_logprob = logp_posterior(proposal, y_pred_test)

    # Compute log acceptance ratio (Metropolis-Hastings)
    acc_logprob = proposal_logprob - curr_logprob
    acc_logprob[acc_logprob > 0] = 0  # Cap at 0 to avoid exp overflow (acceptance prob ≤ 1)

    # Convert to acceptance probabilities
    acc_prob = np.exp(acc_logprob)

    # Decide whether to accept each proposal
    acc = np.random.uniform(size=len(current)) < acc_prob

    # Track acceptance probabilities (not binary outcomes)
    total_acc += acc_prob

    # Update states and log-probs where proposals are accepted
    current[acc] = proposal[acc]
    curr_logprob[acc] = proposal_logprob[acc]

    # Save a copy of the current state
    states.append(np.copy(current)[None])

    # Periodically print progress and acceptance rates
    if i % (0.1*Nstep) == 0.1*Nstep-1:
        print(
            'step:', len(states),
            'Time:', time.time() - t,
            'Min acceptance rate:', np.min(total_acc / (i + 1)),
            'Mean acceptance rate:', np.mean(total_acc / (i + 1))
        )
        t = time.time()  # Reset timer for next print interval

In [ ]:
# remove burn-in
states = np.concatenate(states[int(0.2*Nstep):], 0)

# mean and std of samples
mean = np.mean(states, 0)
errorbar = np.std(states, 0)

In [ ]:
data = {"means": mean.tolist(), "errorbars": errorbar.tolist()}
the_date = datetime.datetime.now().strftime("%y-%m-%d-%H-%M")
zip_file_name = 'Submission_' + the_date + '.zip'
zip_file = Utility.save_json_zip(
    submission_dir="submissions",
    json_file_name="result.json",
    zip_file_name=zip_file_name,
    data=data
)
print(f"Submission ZIP saved at: {zip_file}")